In [ ]:
import glob
import cv2
import numpy as np
import matplotlib.image as mpimg
from matplotlib import pyplot as plt

img1w = glob.glob('/home/jorge/Documents/underwater_welding_arc_bubble/graygray/*.tif') #nova2
for x in range(len(img1w)):
    original = mpimg.imread('/home/jorge/Documents/underwater_welding_arc_bubble/graygray/image%d.tif' % (x))
    
    clahe1 = cv2.createCLAHE(clipLimit=4.0, tileGridSize=(245,245))
    clah1 = clahe1.apply(original)
    bilat1 = cv2.bilateralFilter(clah1,15,35,35)
    blur3 = cv2.bilateralFilter(clah1,15,25,25)
    
    adp_gau = cv2.adaptiveThreshold(bilat1, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, \
                                 cv2.THRESH_BINARY, 11, 2);
    
    a = cv2.Laplacian(blur3, cv2.CV_8U,1,1, 3, cv2.BORDER_REPLICATE);
    res = cv2.addWeighted(adp_gau,0.8,a,-1.5,0)
    limiar5, imgLimiar5 = cv2.threshold(res, 189, 255, cv2.THRESH_BINARY)


    th, im_th = cv2.threshold(imgLimiar5, 200, 255, cv2.THRESH_BINARY_INV);
    brt, br2 = cv2.threshold(adp_gau, 200, 255, cv2.THRESH_BINARY_INV)

    im_floodfill = im_th.copy()
    h, w = im_th.shape[:2]
    mask = np.zeros((h+2, w+2), np.uint8)
    cv2.floodFill(im_floodfill, mask, (0,0), 255)
    im_floodfill_inv = cv2.bitwise_not(im_floodfill)
    im_out = br2 | im_floodfill_inv
    im_floodfill2 = im_out.copy()
    h, w = im_th.shape[:2]
    mask = np.zeros((h+2, w+2), np.uint8)
    cv2.floodFill(im_floodfill2, mask, (0,0), 255)
    im_floodfill_inv2 = cv2.bitwise_not(im_floodfill2) 

    im_out2 = im_out | im_floodfill_inv2

    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5))
    kernel1 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3))
    open1 = cv2.morphologyEx(im_out2, cv2.MORPH_OPEN, kernel)
    open12 = cv2.erode(open1,kernel1,iterations = 1)

    bgr = cv2.cvtColor(open12, cv2.COLOR_GRAY2BGR)
    
    contours, hierarchy = cv2.findContours(open12, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    print('areas de imagen')

    for contours in contours:
        area = cv2.contourArea(contours)
        
        if area > 1000:
            cv2.drawContours(bgr, contours, -1, (0,255,0), 3)
            M = cv2.moments(contours)
            perimeter = cv2.arcLength(contours,True)
            print(area)
            #print(M)
            print(perimeter)   
            
    #canny
    thr, thresh = cv2.threshold(open1, 220, 255, cv2.THRESH_BINARY_INV)

    canny = cv2.Canny(thresh,100,200)
    bin2, canny1 = cv2.threshold(canny, 150, 255, cv2.THRESH_BINARY_INV)

    
    

    cv2.imwrite('/home/jorge/Documents/hoy si/vamos a ver/contorno_%d.png' % (x),bgr)
    cv2.imwrite('/home/jorge/Documents/hoy si/vamos a ver/canny_%d.png' % (x),canny1)
    #cv2.imwrite('/home/jorge/Documents/hoy si/vamos a ver/final2_%d.png' % (x),imgLimiar5)

    #cv2.imshow('clah3', clah3);
    #cv2.imshow('blur', blur);
    #cv2.imshow('limiar1', imgLimiar);
    #cv2.imshow('limiar2', imgLimiar2);
    #   cv2.imshow('limiar3', imgLimiar4);
    #cv2.imshow('laplaciano', a);
    # cv2.imshow('resta', res);
    
    #plt.title('adjusted', size=30)
    #plt.imshow(adjusted,cmap = 'gray')
    #plt.show()
    #plt.title('limiar3', size=30)
    #plt.imshow(imgLimiar4,cmap = 'gray')
    #plt.show()
    #plt.title('res', size=30)
    #plt.imshow(res,cmap = 'gray')
    #plt.show()
    cv2.waitKey()
    cv2.destroyAllWindows()